In [131]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [132]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [133]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар?  
Зачем он нужен?  
Используя этот товар мы смещаем качество рекомендаций.
В какую сторону?   
Можно ли удалить этот товар?   
Уберите этот товар (**внимание**: это можно сделать разными способами!) и сравните с качеством на семинаре.

- Не вводить его вообще
- Фильтровать выходы (в ручную, ignore_items)
- Разные товары 999999

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [134]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    return recs.tolist()

In [135]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()

popl = data['sales_value'].sum()

popularity['item_sales_weight_percent'] = (popularity['sales_value'] / popl) *100
popularity.head(10)

,item_id,sales_value,item_sales_weight_percent
0,25671,20.94,0.000282
1,26081,0.99,0.000013
2,26093,1.59,0.000021
3,26190,1.54,0.000021
4,26355,1.98,0.000027
5,26426,2.29,0.000031
6,26540,2.79,0.000038
7,26601,7.59,0.000102
8,26636,2.50,0.000034
9,26691,8.37,0.000113


In [136]:
popularity.describe()

,item_id,sales_value,item_sales_weight_percent
count,8.905100e+04,89051.000000,89051.000000
mean,5.115772e+06,83.458481,0.001123
std,5.178973e+06,1628.715079,0.021915
min,2.567100e+04,0.000000,0.000000
25%,9.665830e+05,3.500000,0.000047
50%,1.448516e+06,10.780000,0.000145
75%,9.553042e+06,46.105000,0.000620
max,1.802456e+07,467993.620000,6.296956


In [137]:
popularity_2 = popularity[['item_id', 'item_sales_weight_percent']]

# popularity_2.sort_values('item_sales_weight_percent')
popularity_2 = popularity_2.sort_values(by=['item_sales_weight_percent'], ascending=False)

popularity_2['logarithm_base2'] = 1/np.log2(popularity_2['item_sales_weight_percent'])

# popularity_2['logarithm_base2'] = popularity_2['user_id'].apply(lambda x: popular_recs)
popularity_2.head(10)

C:\Users\Newman\anaconda3\lib\site-packages\pandas\core\arraylike.py:274: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


,item_id,item_sales_weight_percent,logarithm_base2
56233,6534178,6.296956,0.376697
56193,6533889,0.573808,-1.247878
29195,1029743,0.511055,-1.032578
56228,6534166,0.421134,-0.801508
35054,1082185,0.367207,-0.691881
16539,916122,0.360987,-0.680281
56184,6533765,0.352773,-0.665253
37719,1106523,0.346555,-0.654088
25333,995242,0.321509,-0.610848
52184,5569230,0.294336,-0.566744


In [138]:
new_df = data.merge(popularity_2, on="item_id")
# new_df = new_df.sort_values(by=['item_sales_weight_percent'], ascending=False, inplace=True)
new_df.head(5)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,item_sales_weight_percent,logarithm_base2
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,0.117942,-0.32427
1,1929,27021022215,4,1004906,1,1.39,441,-0.6,1755,1,0.0,0.0,0.117942,-0.32427
2,568,27021090189,4,1004906,1,1.39,446,-0.6,35,1,0.0,0.0,0.117942,-0.32427
3,887,27031467254,5,1004906,1,1.39,298,-0.6,1551,1,0.0,0.0,0.117942,-0.32427
4,175,27057508267,7,1004906,1,1.99,446,0.0,2303,2,0.0,0.0,0.117942,-0.32427


In [139]:
sale_weight = new_df.groupby('item_id')['sales_value'].sum().reset_index()
sale_weight.sort_values(by=['sales_value'], ascending=False, inplace=True)
sale_weight = sale_weight.reset_index()
sale_weight = sale_weight.drop(index=[0])
sale_weight

,index,item_id,sales_value
1,56193,6533889,42645.75
2,29195,1029743,37981.91
3,56228,6534166,31298.96
4,35054,1082185,27291.02
5,16539,916122,26828.79
...,...,...,...
89046,50764,2961522,0.00
89047,70059,10121805,0.00
89048,39334,1121502,0.00
89049,5918,821773,0.00


In [140]:


sale_weight.sort_values(by=['sales_value'], ascending=False, inplace=True)
total_sales = sale_weight['sales_value'].sum()
sale_weight['item_sales_weight_percent'] = (sale_weight['sales_value'] / total_sales) *100
sale_weight['logarithm_base2'] = (1/np.log2(sale_weight['item_sales_weight_percent'])) * (-1)
sale_weight = sale_weight.reset_index()

sale_weight.head(10)

C:\Users\Newman\anaconda3\lib\site-packages\pandas\core\arraylike.py:274: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


,level_0,index,item_id,sales_value,item_sales_weight_percent,logarithm_base2
0,1,56193,6533889,42645.75,0.612368,1.413371
1,2,29195,1029743,37981.91,0.545398,1.143357
2,3,56228,6534166,31298.96,0.449435,0.866690
3,4,35054,1082185,27291.02,0.391883,0.739916
4,5,16539,916122,26828.79,0.385246,0.726666
5,6,56184,6533765,26218.33,0.376480,0.709545
6,7,37719,1106523,25756.15,0.369843,0.696858
7,8,25333,995242,23894.76,0.343115,0.647989
8,9,52184,5569230,21875.22,0.314116,0.598576
9,10,8493,844179,18094.23,0.259823,0.514297


In [141]:
sale_weight['weighted_sales_value'] = (sale_weight['sales_value'] * sale_weight['logarithm_base2']) 
sale_weight.head(10)

,level_0,index,item_id,sales_value,item_sales_weight_percent,logarithm_base2,weighted_sales_value
0,1,56193,6533889,42645.75,0.612368,1.413371,60274.273543
1,2,29195,1029743,37981.91,0.545398,1.143357,43426.868740
2,3,56228,6534166,31298.96,0.449435,0.866690,27126.486443
3,4,35054,1082185,27291.02,0.391883,0.739916,20193.075512
4,5,16539,916122,26828.79,0.385246,0.726666,19495.567245
5,6,56184,6533765,26218.33,0.376480,0.709545,18603.078035
6,7,37719,1106523,25756.15,0.369843,0.696858,17948.370936
7,8,25333,995242,23894.76,0.343115,0.647989,15483.543138
8,9,52184,5569230,21875.22,0.314116,0.598576,13093.973517
9,10,8493,844179,18094.23,0.259823,0.514297,9305.814379


In [142]:
sale_weight['item_sales_weight_percent'].sum()

100.00000000000001

In [143]:
sale_weight['logarithm_base2'].sum()

7596.615744936096

In [144]:
sale_weight['weighted_sales_value'].sum()/sale_weight['sales_value'].sum()

0.18366355831704856

In [145]:

final_db  =  data.merge(sale_weight, on="item_id")
final_db.sort_values(by=['weighted_sales_value'], ascending=False, inplace=True)
# final_db.reset_index()

In [146]:
# final_db = final_db.reset_index()
final_db.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value_x,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,level_0,index,sales_value_y,item_sales_weight_percent,logarithm_base2,weighted_sales_value
1667054,1619,30087532488,216,6533889,21774,51.15,422,-2.17,1617,32,0.0,0.0,1,56193,42645.75,0.612368,1.413371,60274.273543
1666956,825,29276758955,172,6533889,9666,29.66,375,-0.01,1805,25,0.0,0.0,1,56193,42645.75,0.612368,1.413371,60274.273543
1666926,1079,29115027663,161,6533889,10046,27.01,318,-0.01,1632,24,0.0,0.0,1,56193,42645.75,0.612368,1.413371,60274.273543
1666925,825,29114820303,161,6533889,9702,26.67,375,-0.01,735,24,0.0,0.0,1,56193,42645.75,0.612368,1.413371,60274.273543
1666924,906,29114799129,161,6533889,8862,24.18,319,-0.89,2210,24,0.0,0.0,1,56193,42645.75,0.612368,1.413371,60274.273543
1666923,1023,29095133403,160,6533889,21982,56.69,429,-2.20,1934,24,0.0,0.0,1,56193,42645.75,0.612368,1.413371,60274.273543
1666922,1609,29095123177,160,6533889,15946,43.84,316,-1.59,1913,24,0.0,0.0,1,56193,42645.75,0.612368,1.413371,60274.273543
1666921,825,29094671633,159,6533889,13401,35.50,375,-1.34,1301,23,0.0,0.0,1,56193,42645.75,0.612368,1.413371,60274.273543
1666920,1982,29094629381,159,6533889,3511,9.76,319,-0.01,2112,23,0.0,0.0,1,56193,42645.75,0.612368,1.413371,60274.273543
1666919,1982,29094625865,159,6533889,11293,31.16,319,-1.13,1153,23,0.0,0.0,1,56193,42645.75,0.612368,1.413371,60274.273543


In [147]:
test_size_weeks = 3

data_train = final_db[final_db['week_no'] < final_db['week_no'].max() - test_size_weeks]
data_test = final_db[data['week_no'] >= final_db['week_no'].max() - test_size_weeks]

C:\Users\Newman\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [148]:
data_train.shape[0], data_test.shape[0]

(2260945, 99950)

In [149]:
data_test.groupby('user_id')['item_id'].unique().reset_index()

,user_id,item_id
0,1,"[15926844, 15926712, 15926927, 13877012, 13511..."
1,2,"[13189726, 858373, 16035203, 13213074, 1377698..."
2,3,"[13842214, 13945244, 15596279, 15741861, 13671..."
3,4,"[13115703, 13115481, 9297031, 10121570]"
4,5,"[13159269, 13987103, 13986893, 13986405, 13212..."
...,...,...
2379,2496,"[13945244, 13910296, 13382073, 15506577, 15452..."
2380,2497,"[15831255, 994685, 13511457, 13040302, 1545281..."
2381,2498,"[13841744, 8011291, 13008079, 13910296, 159269..."
2382,2499,"[13842090, 13945244, 13382073, 15511891, 13512..."


In [150]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result['actual'] = result['actual'].apply(lambda x: list(x))
result.head(10)

,user_id,actual
0,1,"[15926844, 15926712, 15926927, 13877012, 13511..."
1,2,"[13189726, 858373, 16035203, 13213074, 1377698..."
2,3,"[13842214, 13945244, 15596279, 15741861, 13671..."
3,4,"[13115703, 13115481, 9297031, 10121570]"
4,5,"[13159269, 13987103, 13986893, 13986405, 13212..."
5,6,"[15926712, 12692332, 15926863, 14019783, 13007..."
6,7,"[8011291, 13040302, 1110039, 15511891, 1311551..."
7,8,"[13189726, 15926886, 15717170, 1118963, 155965..."
8,9,"[13671735, 16729296]"
9,10,"[15596488, 1085653]"


In [151]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом датасете {} юзеров'.format(test_users))
print('В тестовом датасете {} новых юзеров'.format(new_test_users))

В тестовом датасете 2384 юзеров
В тестовом датасете 0 новых юзеров


In [39]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_weight_percent'].sum().reset_index()
    popular.sort_values('logarithm_base2', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [40]:
popularity_recommendation(popular, n=5)

KeyError: 'logarithm_base2'

In [11]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)

result.head(2)

[6534178,
 1029743,
 6533889,
 1106523,
 6533765,
 5569230,
 1082185,
 6534166,
 844179,
 916122,
 995242,
 12484608,
 5569471,
 1070820,
 1126899,
 878996,
 899624,
 866211,
 1089093,
 981760,
 951590,
 1070702,
 1133018,
 1004906,
 1135476,
 863447,
 1005186,
 854852,
 1098844,
 8090537,
 962568,
 986912,
 828106,
 883404,
 874972,
 1081177,
 859075,
 914190,
 923746,
 1044078,
 5568378,
 6034857,
 9245106,
 1075368,
 1134688,
 5569845,
 961554,
 903325,
 8090521,
 854405,
 12810391,
 913210,
 844165,
 1068719,
 12810393,
 840361,
 976199,
 999858,
 12812261,
 1058997,
 908531,
 947858,
 862349,
 12731432,
 849843,
 880150,
 901062,
 1110843,
 845208,
 970747,
 1096036,
 1022254,
 1051323,
 1053690,
 878715,
 948670,
 1055646,
 1069506,
 973916,
 12757087,
 12811532,
 995785,
 849202,
 1101010,
 933835,
 860776,
 1127831,
 1026118,
 949023,
 1034176,
 979707,
 15972074,
 994685,
 15926886,
 1138443,
 940947,
 9396835,
 12487897,
 831628,
 5585510,
 839419,
 972418,
 965766,
 909714,


In [76]:
def popularity_recommendation(data, n=5):
    """Топ-n sales_weight товаров"""
    
#     popular = data.groupby('item_sales_weight_percent').reset_index()
    popular = data.groupby('item_id')['item_sales_weight_percent'].sum().reset_index()
    popular.sort_values('item_sales_weight_percent', ascending=False, inplace=True)
    
    recs = popular.head(n)
    
    return recs.tolist()

In [77]:
popularity_recommendation(new_df, n=5)

AttributeError: 'DataFrame' object has no attribute 'tolist'

In [63]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(new_df, n=5)

new_df['popular_recommendation'] = new_df['user_id'].apply(lambda x: popular_recs)
new_df.head(2)

AttributeError: 'NoneType' object has no attribute 'head'

In [48]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
#     recs = np.random.choice(items, size=n, replace=False)
    sale_weight = 
    
    return recs.tolist()

SyntaxError: invalid syntax (<ipython-input-48-72a4bd81f27c>, line 6)

In [ ]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

Сделайте предсказания

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [ ]:
result = pd.read_csv('preds.csv')# закгрузка predict с семианара
result.head(2)

In [ ]:
type(result['actual'][0])

In [ ]:
# файл с предсказаниями сохранил предсказания как строки
# нужно перевести обратно в список
for a in result.columns[1:]:
    result.loc[:, a]=  result[a].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))

In [ ]:
type(result['own_purchases'][0])

In [ ]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [ ]:
result.apply(lambda x: precision_at_k(x['own_purchases'], x['actual'],  5), axis=1).mean()

### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


In [ ]:
# your_code

### Задание 4*. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.